In [1]:
# Adapted from https://github.com/GoogleCloudPlatform/cloudml-samples/blob/master/notebooks/tensorflow/getting-started-keras.ipynb
import os
import sys

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../credentials/sticky-notes1-data-access.json'

### Run package locally with 'python3 -m'

In [20]:
! rm -r ../train-output

# Test that the package works
! python3 -m trainer.task \
    --job-dir ../train-output \
    --data_dir ../../data/processed/tfrecords/yolo-100-examples \
    --epochs 1 \
    --learning_rate=0.001

2020-05-09 13:31:21.990852: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-05-09 13:31:22.019005: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fccbe35dea0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-05-09 13:31:22.019114: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Train for 2 steps, validate for 1 steps
2020-05-09 13:31:52.623199: I tensorflow/core/profiler/lib/profiler_session.cc:225] Profiler session started.
1/2 [==============>...............] - ETA: 29s - loss: 7.0665 - accuracy: 0.5938 - val_loss: 25.8491 - val_accuracy: 0.3438^C


### Run locally with 'gcloud ai-platform local train'

In [16]:
# Explicitly tell `gcloud ai-platform local train` to use Python 3 
! gcloud config set ml_engine/local_python $(which python3)

# Run the trainer with ai-platform local train
! gcloud ai-platform local train \
    --package-path ./trainer \
    --module-name trainer.task \
    --job-dir ../train-output \
    -- \
    --data_dir ../../data/processed/tfrecords/100-examples

Updated property [ml_engine/local_python].
^C


Command killed by keyboard interrupt

2020-05-09 12:57:35.781321: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-05-09 12:57:35.806404: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fdf980edf70 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-05-09 12:57:35.806437: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/julian/projects/sticky-notes/analysis/stickynet/t

### Submit job to gcloud

In [2]:
import datetime

BUCKET_NAME = 'sticky-notes1'
os_variables = {
    'BUCKET_NAME': BUCKET_NAME,
    'REGION': 'europe-west1',
    'JOB_NAME': 'sticky_job',
    'JOB_DIR': 'gs://' + BUCKET_NAME + '/training-jobs',
    'DATA_DIR': 'gs://' + BUCKET_NAME + '/training-data/10000-examples'
}
for key, value in os_variables.items():
    os.environ[key] = value

In [5]:
! export date=$(date +%y%m%d_%H%M%S); \
  gcloud ai-platform jobs submit training "${JOB_NAME}_${date}" \
    --config config.yaml \
    --package-path trainer/ \
    --module-name trainer.task \
    --region $REGION \
    --python-version 3.7 \
    --runtime-version 2.1 \
    --job-dir "${JOB_DIR}/${date}" \
    -- \
    --epochs 16 \
    --data_dir $DATA_DIR \
#    --initial_weights_path gs://sticky-notes1/training-jobs/200506_181220/1/model-weights.tf \
#    --first_epoch 8

Job [sticky_job_200507_124624] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe sticky_job_200507_124624

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs sticky_job_200507_124624
jobId: sticky_job_200507_124624
state: QUEUED


In [ ]:
# view training logs
! tensorboard --logdir $JOB_DIR/

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.1.1 at http://localhost:6006/ (Press CTRL+C to quit)


In [28]:
# MODEL_NAME="mynet"

# ! gcloud ai-platform models create $MODEL_NAME \
#   --regions $REGION

In [29]:
# MODEL_VERSION="v1"

# # Get a list of directories in the `keras_export` parent directory. Then pick
# # the directory with the latest timestamp, in case you've trained multiple
# # times.
# SAVED_MODEL_PATH=$(gsutil ls $JOB_DIR/keras_export | tail -n 1)

# # Create model version based on that SavedModel directory
# gcloud ai-platform versions create $MODEL_VERSION \
#   --model $MODEL_NAME \
#   --runtime-version 1.15 \
#   --python-version 3.7 \
#   --framework tensorflow \
#   --origin $SAVED_MODEL_PATH